In [5]:
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
from cmdstanpy import CmdStanModel
import cmdstanpy
import arviz as az
import pandas as pd
import csv,re

from DA_tools.DA_tools import ribbon_plot
from DA_tools.FDA_data_prepare import create_spline_matrix
from DA_tools.DA_colors import *
import os
plt.style.context("seaborn-white")
mpl.rcParams["figure.dpi"] = 200

In [2]:
LIGHT = "#B3FFFF"  # 179, 255, 255,
LIGHT_HIGHLIGHT = "#9AF6FF"  # 154, 246,255
MID = "#67C3FF"  # 103,195,255
MID_HIGHLIGHT = "#3490CC"  # 52,144,204
DARK = "#015D99"  # 1,93,153
DARK_HIGHLIGHT = "#002A66"  # 0,42,102
GREEN = "#00FF00"  # RGB
LIGHT_GREY = "#DDDDDD"  # RGB

In [9]:
acc_healthy = pd.read_csv('home/data_preprocesed/acc_healthy_samples.csv')
acc_damaged = pd.read_csv('home/data_preprocesed/acc_damaged_samples.csv')
acc_very_damaged = pd.read_csv('home/data_preprocesed/acc_very_damaged_samples.csv')

gyro_healthy = pd.read_csv('home/data_preprocesed/gyro_healthy_samples.csv')
gyro_damaged = pd.read_csv('home/data_preprocesed/gyro_damaged_samples.csv')
gyro_very_damaged = pd.read_csv('home/data_preprocesed/gyro_very_damaged_samples.csv')

gyro_agg_healthy = pd.read_csv('home/data_preprocesed/gyro_agg_healthy_samples.csv')
gyro_agg_damaged = pd.read_csv('home/data_preprocesed/gyro_agg_damaged_samples.csv')
gyro_agg_very_damaged = pd.read_csv('home/data_preprocesed/gyro_agg_very_damaged_samples.csv')

In [15]:
acc_healthy_data = np.array([acc_healthy[col].values for col in acc_healthy.columns if col.startswith('Sample')])
acc_damaged_data = np.array([acc_damaged[col].values for col in acc_damaged.columns if col.startswith('Sample')])
acc_v_damaged_data = np.array([acc_very_damaged[col].values for col in acc_very_damaged.columns if col.startswith('Sample')])
acc = [acc_healthy_data,acc_damaged_data,acc_v_damaged_data]

gyro_healthy_data = np.array([gyro_healthy[col].values for col in gyro_healthy.columns if col.startswith('Sample')])
gyro_damaged_data = np.array([gyro_damaged[col].values for col in gyro_damaged.columns if col.startswith('Sample')])
gyro_very_damaged_data = np.array([gyro_very_damaged[col].values for col in gyro_very_damaged.columns if col.startswith('Sample')])
gyro = [gyro_healthy_data,gyro_damaged_data,gyro_very_damaged_data]

gyro_agg_healthy_data = np.array([gyro_agg_healthy[col].values for col in gyro_agg_healthy.columns if col.startswith('Sample')])
gyro_agg_damaged_data = np.array([gyro_agg_damaged[col].values for col in gyro_agg_damaged.columns if col.startswith('Sample')])
gyro_agg_very_damaged_data = np.array([gyro_agg_very_damaged[col].values for col in gyro_agg_very_damaged.columns if col.startswith('Sample')])
gyro_agg = [gyro_agg_healthy_data,gyro_agg_damaged_data,gyro_agg_very_damaged_data]

In [12]:
def prepare_data(data_array,spl_order = 3, num_knots = 30, frequencies = None, mode = 'binary', training_samples = 5,lambda0=None):
    N = len(data_array[0][0])
    spl_order = spl_order
    num_knots = num_knots
    if frequencies is None:
        times = np.linspace(0,N*10,N)
        knot_list = np.quantile(times,np.linspace(0,1,num_knots))
        B0 = create_spline_matrix(N, times, spl_order, num_knots)

    else:
        knot_list = np.quantile(frequencies,np.linspace(0,1,num_knots))
        B0 = create_spline_matrix(N, frequencies, spl_order, num_knots)

    K = num_knots+2
    if mode == 'binary':
        M = 2
        IL1 = training_samples
        IL2 = training_samples
        IL = IL1+IL2

        num_healthy = len(data_array[0])
        num_damaged = len(data_array[1])+len(data_array[2])
        data_array_damaged = np.concatenate([data_array[1],data_array[2]])
        total = num_healthy + num_damaged
        IT = total - IL

        sampling_order_1 = np.random.permutation([*range(num_healthy)])
        sampling_order_2 = np.random.permutation([*range(num_damaged)])

        y_labeled = np.concatenate(
        [np.array(data_array[0])[sampling_order_1[:IL1]],
            np.array(data_array_damaged)[sampling_order_2[:IL2]]])
        y_labeled = y_labeled.T
        labels = np.concatenate([np.ones(IL1), 2*np.ones(IL2)]).astype(int)


        y_test = np.concatenate(
            [np.array(data_array[0])[sampling_order_1[IL1:]],
                np.array(data_array_damaged)[sampling_order_2[IL2:]]
                ]
        )
        y_test = y_test.T
        y_test_labels = np.concatenate([np.ones(num_healthy-IL1), 2*np.ones(num_damaged-IL2)]).astype(int)
        if lambda0 is None:
            lambda0 = np.array([(IL1)/(IL), (IL2)/(IL)])

        IT = y_test.shape[1]

    if mode == 'all':
        M = 3
        IL1 = training_samples
        IL2 = training_samples
        IL3 = training_samples
        IL = IL1+IL2+IL3

        num_healthy = len(data_array[0])
        num_damaged = len(data_array[1])
        num_very_damaged = len(data_array[2])

        total = num_healthy + num_damaged + num_very_damaged
        IT = total - IL

        sampling_order_1 = np.random.permutation([*range(num_healthy)])
        sampling_order_2 = np.random.permutation([*range(num_damaged)])
        sampling_order_3 = np.random.permutation([*range(num_very_damaged)])

        y_labeled = np.concatenate(
        [np.array(data_array[0])[sampling_order_1[:IL1]],
            np.array(data_array[1])[sampling_order_2[:IL2]],
            np.array(data_array[2])[sampling_order_3[:IL3]],
            ])
        y_labeled = y_labeled.T
        labels = np.concatenate([np.ones(IL1), 2*np.ones(IL2), 3*np.ones(IL3)]).astype(int)


        y_test = np.concatenate(
            [np.array(data_array[0])[sampling_order_1[IL1:]],
                np.array(data_array[1])[sampling_order_2[IL2:]],
                np.array(data_array[2])[sampling_order_3[IL3:]]
                ]
        )
        y_test = y_test.T
        y_test_labels = np.concatenate([np.ones(num_healthy-IL1), 2*np.ones(num_damaged-IL2),3*np.ones(num_very_damaged-IL3)]).astype(int)
        if lambda0 is None:
            lambda0 = np.array([(IL1)/(IL), (IL2)/(IL), (IL3)/(IL)])

        IT = y_test.shape[1]

    data_out = {
    "N": N,
    "IL": IL,
    "K": K,
    "M": M,
    "x": B0,
    "labels": labels,
    "y_labeled": y_labeled,
    "lambda0": lambda0,
    "IT": IT,
    "y_test": y_test,
    }

    return data_out,y_test_labels,IT,IL, total
    

def get_results(model, data, seed, labels, IT, IL, total, mode = 'binary'):
    if mode == 'binary':
        result = model.sample(data=data, seed=seed)
        probs_from_arviz = az.summary(
        result, "log_probabilities", kind='stats', round_to=5)
        probs_from_arviz_p = az.summary(
            result, "probabilities", kind='stats', round_to=5)
            
        indices_cat1 = labels == 1
        indices_cat2 = labels == 2

        cat1 = probs_from_arviz.iloc[:IT, :].iloc[indices_cat1, :]
        cat2 = probs_from_arviz.iloc[IT:2*IT, :].iloc[indices_cat2, :]

        cat1p = probs_from_arviz_p.iloc[:IT, :].iloc[indices_cat1, :]
        cat2p = probs_from_arviz_p.iloc[IT:2*IT, :].iloc[indices_cat2, :]


        a = sum((cat1["mean"].values) < np.log(0.5))
        b = sum((cat2["mean"].values) < np.log(0.5))

        hit_rate = 1 - (a+b)/(total-IL)
        print('hit rate = ',hit_rate)
    if mode == 'all':
        result = model.sample(data=data, seed=seed)
        probs_from_arviz = az.summary(
        result, "log_probabilities", kind='stats', round_to=5)
        probs_from_arviz_p = az.summary(
            result, "probabilities", kind='stats', round_to=5)
            
        indices_cat1 = labels == 1
        indices_cat2 = labels == 2
        indices_cat3 = labels == 2

        cat1 = probs_from_arviz.iloc[:IT, :].iloc[indices_cat1, :]
        cat2 = probs_from_arviz.iloc[IT:2*IT, :].iloc[indices_cat2, :]
        cat3 = probs_from_arviz.iloc[2*IT:3*IT, :].iloc[indices_cat3, :]

        cat1p = probs_from_arviz_p.iloc[:IT, :].iloc[indices_cat1, :]
        cat2p = probs_from_arviz_p.iloc[IT:2*IT, :].iloc[indices_cat2, :]
        cat3p = probs_from_arviz_p.iloc[2*IT:3*IT, :].iloc[indices_cat3, :]


        a = sum((cat1["mean"].values) < np.log(0.5))
        b = sum((cat2["mean"].values) < np.log(0.5))
        c = sum((cat3["mean"].values) < np.log(0.5))

        hit_rate = 1 - (a+b+c)/(total-IL)
        print('hit rate = ',hit_rate)
        




In [26]:
frequencies = gyro_damaged['Frequencies']
# acc, gyro, gyro_agg
data, labels, IT, IL, total = prepare_data(acc,frequencies=None,training_samples=5,spl_order = 3, num_knots=15, mode = 'binary')
model = CmdStanModel(stan_file='home/stan/mix.stan')
seed = 26042024
get_results(model=model,data=data,seed=seed,labels=labels,IT=IT,IL=IL,total=total,mode='binary')

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |          | 00:00 Status


chain 1 |▉         | 00:00 Iteration:    1 / 2000 [  0%]  (Warmup)


chain 1 |█▊        | 00:00 Iteration:  200 / 2000 [ 10%]  (Warmup)


chain 1 |███▏      | 00:00 Iteration:  500 / 2000 [ 25%]  (Warmup)


chain 1 |████▌     | 00:00 Iteration:  800 / 2000 [ 40%]  (Warmup)


chain 1 |█████▉    | 00:00 Iteration: 1001 / 2000 [ 50%]  (Sampling)


chain 1 |██████▊   | 00:02 Iteration: 1200 / 2000 [ 60%]  (Sampling)



chain 1 |███████▋  | 00:03 Iteration: 1400 / 2000 [ 70%]  (Sampling)




chain 1 |████████▏ | 00:04 Iteration: 1500 / 2000 [ 75%]  (Sampling)


chain 1 |████████▋ | 00:05 Iteration: 1600 / 2000 [ 80%]  (Sampling)


chain 1 |█████████ | 00:06 Iteration: 1700 / 2000 [ 85%]  (Sampling)


chain 1 |█████████▌| 00:07 Iteration: 1800 / 2000 [ 90%]  (Sampling)


chain 1 |██████████| 00:07 Sampling completed                       
chain 2 |██████████| 00:07


INFO:cmdstanpy:CmdStan done processing.



hit rate =  0.9642857142857143


In [27]:
result = model.sample(data=data, seed=seed)
probs_from_arviz = az.summary(
result, "log_probabilities", kind='stats', round_to=5)
probs_from_arviz_p = az.summary(
    result, "probabilities", kind='stats', round_to=5)
    
indices_cat1 = labels == 1
indices_cat2 = labels == 2

cat1 = probs_from_arviz.iloc[:IT, :].iloc[indices_cat1, :]
cat2 = probs_from_arviz.iloc[IT:2*IT, :].iloc[indices_cat2, :]

cat1p = probs_from_arviz_p.iloc[:IT, :].iloc[indices_cat1, :]
cat2p = probs_from_arviz_p.iloc[IT:2*IT, :].iloc[indices_cat2, :]


a = sum((cat1["mean"].values) < np.log(0.5))
b = sum((cat2["mean"].values) < np.log(0.5))

hit_rate = 1 - (a+b)/(total-IL)
print('hit rate = ',hit_rate)

INFO:cmdstanpy:CmdStan start processing
chain 1 |          | 00:00 Status





chain 1 |█▎        | 00:00 Iteration:  100 / 2000 [  5%]  (Warmup)


chain 1 |██▎       | 00:00 Iteration:  300 / 2000 [ 15%]  (Warmup)


chain 1 |███▏      | 00:00 Iteration:  500 / 2000 [ 25%]  (Warmup)


chain 1 |████▌     | 00:00 Iteration:  800 / 2000 [ 40%]  (Warmup)




chain 1 |█████▉    | 00:01 Iteration: 1001 / 2000 [ 50%]  (Sampling)


chain 1 |██████▊   | 00:02 Iteration: 1200 / 2000 [ 60%]  (Sampling)


chain 1 |███████▎  | 00:03 Iteration: 1300 / 2000 [ 65%]  (Sampling)



chain 1 |███████▋  | 00:04 Iteration: 1400 / 2000 [ 70%]  (Sampling)

chain 1 |████████▏ | 00:05 Iteration: 1500 / 2000 [ 75%]  (Sampling)
chain 1 |████████▋ | 00:05 Iteration: 1600 / 2000 [ 80%]  (Sampling)


chain 1 |█████████ | 00:06 Iteration: 1700 / 2000 [ 85%]  (Sampling)


chain 1 |█████████▌| 00:07 Iteration: 1800 / 2000 [ 90%]  (Sampling)




chain 1 |██████████| 00:08 Sampling completed                       
chain 2 |██████████| 00:08 Sampling completed

chain 3 |██████████| 00:08 Sampling co


INFO:cmdstanpy:CmdStan done processing.



hit rate =  0.9642857142857143


In [34]:
thetas = result.stan_variables()


In [29]:
cat2p

,mean,sd,hdi_3%,hdi_97%
"probabilities[1, 17]",0.99946,0.01823,1.00000,1.00000
"probabilities[1, 18]",1.00000,0.00000,1.00000,1.00000
"probabilities[1, 19]",0.97324,0.12495,0.94071,1.00000
"probabilities[1, 20]",1.00000,0.00000,1.00000,1.00000
"probabilities[1, 21]",0.29970,0.39148,0.00000,0.99803
"probabilities[1, 22]",0.99532,0.05688,0.99990,1.00000
"probabilities[1, 23]",1.00000,0.00000,1.00000,1.00000
"probabilities[1, 24]",1.00000,0.00000,1.00000,1.00000
"probabilities[1, 25]",1.00000,0.00000,1.00000,1.00000
"probabilities[1, 26]",1.00000,0.00000,1.00000,1.00000
